In [ ]:
# instalação do Weights & Biases
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import wandb
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from joblib import dump, load

In [ ]:
# wandb login
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# 1 - Pegando do wandb os arquivos necessários para o teste

In [ ]:
# inicializando uma run no wandb
run = wandb.init(project="Breast Cancer TCC", job_type="test_model")

wandb: Currently logged in as: valmirfrancisco1. Use `wandb login --relogin` to force relogin


In [ ]:
# baixando a class necessária para o pipeline funcionar
artifact = run.use_artifact('valmirfrancisco1/Breast Cancer TCC/imageProcessor_class:latest', type='python')
artifact_dir = artifact.download()
file_path = f"{artifact_dir}/imageProcessor.py"
!mv {file_path} /content/

wandb:   1 of 1 files downloaded.  


In [ ]:
# importação da classe
from imageProcessor import ImageProcessor

In [ ]:
# baixando o pipeline
artifact = run.use_artifact('valmirfrancisco1/Breast Cancer TCC/animation_model_pipeline:latest', type='pipeline')
loaded_pipeline = artifact.file()
# carregando o pipeline em uma variável
pipeline = load(loaded_pipeline)

# 2 - Importar o dataset separado para teste

In [ ]:
artifact = run.use_artifact('valmirfrancisco1/Breast Cancer TCC/label_data_test:latest', type='split_data').file()
artifact2 = run.use_artifact('valmirfrancisco1/Breast Cancer TCC/img_data_test:latest', type='split_data').file()
labels = pd.read_csv(artifact)
labels = labels["cancer"]
data_img = np.load(artifact2)

# 3 - Teste

In [ ]:
predictions = pipeline.predict(data_img)

15/15 [==============================] - 13s 358ms/step


In [ ]:
predict = []
for i in predictions:
    predict.append(np.argmax(i))

In [ ]:
print(classification_report(labels, predict))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67       232
           1       0.00      0.00      0.00       232

    accuracy                           0.50       464
   macro avg       0.25      0.50      0.33       464
weighted avg       0.25      0.50      0.33       464



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 4 - Teste com imagens pegas na net

Não vou usar isso aqui por enquanto!!!

In [ ]:
from PIL import Image

# Exemplo de uso:
image1 = Image.open("/content/anime.png")
image2 = Image.open("/content/cartoon.jpg")

# Adicione quantas imagens desejar ao pipeline
images = [image1, image2]

processed_images = pipeline.predict(images)
binary_array = (processed_images >= 0.5).astype(int)

for i, index in enumerate(range(len(images))):
    plt.figure()
    if(binary_array[index][0] == 1):
      plt.title('Anime')
    else:
      plt.title('Cartoon')

    plt.imshow(images[i])

FileNotFoundError: [Errno 2] No such file or directory: '/content/anime.png'

In [ ]:
run.finish()